In [23]:
import numpy as np
from data import Lattice, Catalogue
from utils import plotting
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
cat = Catalogue.from_file('./filt_wind.lat', 0)
cat

Unit cell catalogue with 8301 entries

In [5]:
df = pd.DataFrame(cat)
# df['num_edges'] = df['edge_adjacency'].apply(len)
# df['num_nodes'] = df['nodal_positions'].apply(len)
df['num_edges'] = df['fundamental_edge_adjacency'].apply(len)
df['num_nodes'] = df['fundamental_edge_adjacency'].apply(lambda x: len(np.unique(x)))
df.head()

,name,lattice_constants,nodal_positions,edge_adjacency,fundamental_edge_adjacency,fundamental_tesselation_vecs,num_edges,num_nodes
0,cub_Z06.0_E1,"[1.0, 1.0, 1.0, 90.0, 90.0, 90.0]","[[0.0, 0.77636, 0.18995], [0.5235, 0.0, 0.1899...","[[0, 3], [1, 3], [2, 3], [3, 4], [3, 5], [3, 6]]","[[3, 3], [3, 3], [3, 3]]","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0....",3,1
1,ort_Z08.0_E2,"[0.85075, 0.57449, 1.0, 90.0, 90.0, 90.0]","[[0.0, 0.03354, 0.01299], [0.0, 0.08461, 0.012...","[[0, 3], [1, 9], [2, 5], [4, 5], [5, 7], [5, 9...","[[9, 5], [9, 9], [9, 5], [5, 9], [5, 9], [9, 9]]","[[0.0, 0.0, 0.0, 1.0, -1.0, 0.0], [0.0, 0.0, 0...",6,2
2,cub_Z08.0_E3,"[1.0, 1.0, 1.0, 90.0, 90.0, 90.0]","[[0.0, 0.04173, 0.28413], [0.0, 0.04173, 0.610...","[[0, 4], [1, 5], [2, 8], [3, 8], [6, 8], [7, 8...","[[17, 8], [17, 8], [17, 8], [17, 8], [8, 17], ...","[[0.0, 0.0, 0.0, 1.0, -1.0, 1.0], [0.0, 0.0, 0...",8,2
3,hex_Z08.0_E4,"[0.99471, 0.99471, 1.0, 90.0, 90.0, 120.0]","[[0.0, 0.13048, 0.86004], [0.0, 0.43332, 0.860...","[[0, 4], [1, 4], [2, 4], [3, 4], [4, 5], [4, 6...","[[4, 4], [4, 4], [4, 4], [4, 4]]","[[0.0, 0.0, 0.0, 1.0, 1.0, 0.0], [0.0, 0.0, 0....",4,1
4,tet_Z04.0_E5,"[0.503, 0.503, 1.0, 90.0, 90.0, 90.0]","[[0.0, 0.68023, 0.15914], [0.11139, 0.0, 0.659...","[[0, 3], [1, 4], [2, 3], [3, 4], [3, 7], [4, 5...","[[3, 3], [4, 4], [4, 3], [3, 4]]","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0....",4,2


In [6]:
df.describe()

,num_edges,num_nodes
count,8301.000000,8301.000000
mean,51.428141,23.600891
std,51.274866,25.556775
min,3.000000,1.000000
25%,22.000000,9.000000
50%,36.000000,16.000000
75%,64.000000,28.000000
max,816.000000,384.000000


By using fundamental representation, the number of nodes in a resulting graph is reducing.

In [20]:
# fig = go.Figure()
# fig = make_subplots(
#     rows=1, cols=2, 
#     subplot_titles=("Number of nodes", "Number of edges", "Fundamental nodes", "Fundamental edges")
# )
marker_dict = {'line':{'color':'black', 'width':0.5}}

filt = (df['num_nodes']<200) & (df['num_edges']<200)
# fig.add_histogram(
#     x=df['num_nodes'], histnorm='probability density',
#     name='Nodes', marker=marker_dict
# )
# fig.add_histogram(
#     x=df['num_edges'], histnorm='probability density',
#     name='Edges', marker=marker_dict, 
# )

fig = px.histogram(df.loc[filt,:], x='num_nodes',nbins=50)

fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.75)
# fig.update_layout(xaxis_range=[0,250])
fig.update_layout(title='Unit cell statistics')
fig

In [22]:
plt.

,name,lattice_constants,nodal_positions,edge_adjacency,fundamental_edge_adjacency,fundamental_tesselation_vecs,num_edges,num_nodes
0,cub_Z06.0_E1,"[1.0, 1.0, 1.0, 90.0, 90.0, 90.0]","[[0.0, 0.77636, 0.18995], [0.5235, 0.0, 0.1899...","[[0, 3], [1, 3], [2, 3], [3, 4], [3, 5], [3, 6]]","[[3, 3], [3, 3], [3, 3]]","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0....",3,1
3,hex_Z08.0_E4,"[0.99471, 0.99471, 1.0, 90.0, 90.0, 120.0]","[[0.0, 0.13048, 0.86004], [0.0, 0.43332, 0.860...","[[0, 4], [1, 4], [2, 4], [3, 4], [4, 5], [4, 6...","[[4, 4], [4, 4], [4, 4], [4, 4]]","[[0.0, 0.0, 0.0, 1.0, 1.0, 0.0], [0.0, 0.0, 0....",4,1
7068,cub_Z06.0_R399,"[1.0, 1.0, 1.0, 90.0, 90.0, 90.0]","[[0.0, 0.72073, 0.90465], [0.87334, 0.0, 0.904...","[[0, 3], [1, 3], [2, 3], [3, 4], [3, 5], [3, 6]]","[[3, 3], [3, 3], [3, 3]]","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0....",3,1
7242,hex_Z08.0_R626,"[0.99471, 0.99471, 1.0, 90.0, 90.0, 120.0]","[[0.0, 0.27963, 0.70269], [0.0, 0.36843, 0.702...","[[0, 6], [1, 3], [2, 6], [4, 6], [5, 6], [6, 7...","[[6, 6], [6, 6], [6, 6], [6, 6]]","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0....",4,1
